# Betting on NBA

In [1]:
#TO IMPROVE
# min à 0 lorsque joue pas
# Prendre en compte blessure top7 joueurs - flag nb match avec sans gros joueur
# Poids sur les derniers matchs
# https://www.espn.com/nba/injuries

# Lonnie Walker IV ?

# Prendre l'equipe en face en compte, nombre de points de rebonds, etc.
# 3 points tir 
# flag if injuries last match

# Ne pas prendre en compte si un joueur a joué moins
#Moyenne au llieu du minimum sera plus significatif

#get notif fro new odds unibet Winamax, Betclic
#Problem two Jackson counted twice because they played in the same match Grinzzlies vs LAC

In [3143]:
#pip install selenium
#pip install webdriver-manager
#pip install BeautifulSoup4
import pandas as pd
import numpy as np
import re
import requests
import webbrowser
import seaborn as sns
import urllib.request, json 
import unicodedata
import requests
import math
import csv
import os

from unidecode import unidecode
from IPython.core.display import display, HTML
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
from datetime import datetime, date, timedelta
from tqdm import tqdm

pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
#driver = webdriver.Chrome("C:/Users/Valentin/Documents/Winamax/chromedriver")
driver = webdriver.Chrome(ChromeDriverManager().install())

#Function Soup
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        do = "Nothing"
 



====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST driver version for 110.0.5481
Driver [C:\Users\Valentin\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache


![title](img/UNIBET.png)

In [3167]:
url = "https://www.unibet.fr/zones/v3/sportnode/markets.json?nodeId=2243762&filter=R%C3%A9sultat&marketname=R%C3%A9sultat%20du%20match"
response = requests.get(url)
json_data = response.json()

#Missing:
# (dfGames["competitionName"].str.contains("Norvege")) |
# (dfGames["competitionName"].str.contains("Suède, Allsvenskan")) |

row_data1 = [
    {
        "Day": day,
        "Match": e["eventName"],
        "EventId": e["eventId"],
        "competitionName": e["competitionName"],
    }
    for day, d in enumerate(json_data["marketsByType"][0]["days"])
    for e in d["events"]
]

dfGames = pd.DataFrame(row_data1)
dfGames = dfGames[(dfGames["Day"] >= -99) & ((dfGames["competitionName"].str.contains("Italie, Serie ")) |
                                            (dfGames["competitionName"].str.contains("Allemagne, Bundesliga")) |
                                            (dfGames["competitionName"].str.contains("France, Ligue")) |
                                            (dfGames["competitionName"].str.contains("Espagne, LaLiga")) |
                                            (dfGames["competitionName"].str.contains("Angleterre, Championship")) |
                                            (dfGames["competitionName"].str.contains("Angleterre, Premier League")) |
                                            (dfGames["competitionName"].str.contains("Portugal, Primeira Liga")) | #Portugal, Segunda Liga - Stats not available
                                            (dfGames["competitionName"].str.contains("Danemark, SuperLiga")) |
                                            (dfGames["competitionName"].str.contains("Ecosse, Premiership")) |
                                            (dfGames["competitionName"].str.contains("Autriche, Bundesliga")) |
                                            (dfGames["competitionName"].str.contains("Suisse, Super League")) |
                                            (dfGames["competitionName"].str.contains("Belgique, Pro League")) |
                                            (dfGames["competitionName"].str.contains("Pays-Bas, Eredivisie")) |
                                            (dfGames["competitionName"].str.contains("Turquie, Süper Lig")) |
                                            (dfGames["competitionName"].str.contains("Grèce, Super League")) |
                                            (dfGames["competitionName"].str.contains("Roumanie, Liga 1")) |
                                            (dfGames["competitionName"].str.contains("Israël, Premier League"))
                                          )
                 ]
#dfGames
df = pd.DataFrame(dfGames['Match'].str.split(' - ', expand=True))
df.columns = ['Home', 'Away']
dfGames = pd.concat([df, dfGames], axis=1)
#dfGames
#dfGames.head(20)
dfGames['competitionName'].unique()

array(['Allemagne, Bundesliga 2', 'Danemark, SuperLiga',
       'Grèce, Super League', 'Roumanie, Liga 1', 'Pays-Bas, Eredivisie',
       'Italie, Serie B', 'Autriche, Bundesliga', 'Allemagne, Bundesliga',
       'Belgique, Pro League', 'Espagne, LaLiga2', 'Espagne, LaLiga',
       'Angleterre, Premier League', 'France, Ligue 1 UberEATS®',
       'Portugal, Primeira Liga', 'Angleterre, Championship',
       'Israël, Premier League', 'France, Ligue 2 BKT®',
       'Ecosse, Premiership', 'Turquie, Süper Lig', 'Italie, Serie A',
       'Suisse, Super League'], dtype=object)

In [ ]:
#Get match odds
import datetime
row_data0 = []
nbGames = 0

for idgame in tqdm(dfGames["EventId"]):
    g = "https://www.unibet.fr/zones/event.json?eventId=" + idgame
    with urllib.request.urlopen(g) as url:
        json_data = json.loads(url.read().decode())
        
    competitionName = json_data["eventHeader"]['competitionName']
    timestamp = datetime.datetime.fromtimestamp(json_data["eventHeader"]['eventStartDate']/1000)
    DateTime = timestamp.strftime('%Y-%m-%d %H:%M:%S')
    #home = json_data["eventHeader"]['homeName']
    #away = json_data["eventHeader"]['awayName']
    #i = -1
    if len(json_data['marketClassList'])>1:
        duel = json_data['marketClassList'][1]['marketList'][0]['selections'][1]['name']
        if (" ou ") in json_data['marketClassList'][1]['marketList'][0]['selections'][1]['name']:
            home = strip_accents(duel.split(" ou ")[0])
            away = strip_accents(duel.split(" ou ")[1])
        elif (" Or ") in json_data['marketClassList'][1]['marketList'][0]['selections'][1]['name']:
            home = strip_accents(duel.split(" Or ")[0])
            away = strip_accents(duel.split(" Or ")[1])
        else:
            print("-------------- Problem -------------- " + duel)

        for m in json_data['marketClassList']:
            #i = i + 1
            
            if(m['marketName'] == 'Total de buts' or
               m['marketName'] == 'Combo chance double & Total de buts'):
                for x in m['marketList']:
                    #if(m['marketName'] == 'Combo chance double & Total de buts'):
                        #print(m['marketList'][0]['selections'][0]['name'])
                    for j in x['selections']:
                        data_item = {"IdGame": idgame,
                                     "DateTime": DateTime,
                                     "Competition": competitionName,
                                     "Home": home,
                                     "Away": away,
                                     "Bet": j['name'],
                                     "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                    }
                        row_data0.append(data_item)
            if(m['marketName'] == 'But pour les 2 équipes'):
                for j in m['marketList'][0]['selections']:
                    data_item = {"IdGame": idgame,
                                 "DateTime": DateTime,
                                 "Competition": competitionName,
                                 "Home": home,
                                 "Away": away,
                                 "Bet": j['name'],
                                 "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                }
                    row_data0.append(data_item)
            if(m['marketName'] == 'Résultat du match' or
               m['marketName'] == 'Chance double' or
               m['marketName'] == 'Combo chance double & les 2 équipes marquent'):
                for j in m['marketList'][0]['selections']:
                    data_item = {"IdGame": idgame,
                                 "DateTime": DateTime,
                                 "Competition": competitionName,
                                 "Home": home,
                                 "Away": away,
                                 "Bet": j['name'],
                                 "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                }
                    row_data0.append(data_item)

 29%|██▊       | 77/270 [00:49<01:12,  2.65it/s]

In [ ]:
import datetime
OddGames = pd.DataFrame(row_data0)
next_sunday = datetime.date.today() + datetime.timedelta(days=(6 - today.weekday()) % 7 + 1)
OddGames = OddGames[OddGames['DateTime'] < next_sunday.strftime('%Y-%m-%d 23:59:59')]
OddGames["DateTime"].unique()

Check = pd.DataFrame()
OddGames['Bet'] = OddGames['Bet'].apply(unidecode)
OddGames['Home'] = OddGames['Home'].apply(unidecode)
OddGames['Away'] = OddGames['Away'].apply(unidecode)
Check['BetOri'] = OddGames['Bet']

%run ./MatchTeamNames.ipynb
OddGames["Bet"].replace(special_dict, inplace=True, regex=True)
inverted_dict = {v: k for k, v in merged_dict.items()}
OddGames["Away"].replace(inverted_dict, inplace=True)
OddGames["Home"].replace(inverted_dict, inplace=True)
OddGames["Bet"].replace(inverted_dict, inplace=True)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Away"], 'A') if (p["Away"] in p["Bet"]) else p["Bet"], axis=1)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Home"], 'H') if (p["Home"] in p["Bet"]) else p["Bet"], axis=1)

OddGames["Away"].replace(merged_dict, inplace=True)
OddGames["Home"].replace(merged_dict, inplace=True)
OddGames["Bet"].replace(merged_dict, inplace=True)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Away"], 'A') if (p["Away"] in p["Bet"]) else p["Bet"], axis=1)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Home"], 'H') if (p["Home"] in p["Bet"]) else p["Bet"], axis=1)

OddGames["Bet"] = OddGames["Bet"].str.replace(r'Moins de ', 'M', regex=True).replace(r'Plus de ', 'P', regex=True).replace(r' ou ', '/', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'under ', 'M', regex=True).replace(r'over ', 'P', regex=True).replace(r' Or ', '/', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'Match nul', 'D', regex=True).replace(r'match nul', 'D', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'Draw', 'D', regex=True)
OddGames["Bet"] = OddGames["Bet"].replace(r'Oui', 'LDEM', regex=True).replace(r'Non', 'LDEMP', regex=True).replace(r' ', '', regex=True)
OddGames["Bet"] = OddGames["Bet"].replace(r'Yes', 'LDEM', regex=True).replace(r'No', 'LDEMP', regex=True).replace(r' ', '', regex=True)
OddGames['Bet'] = OddGames['Bet'].str.replace('.', ',', regex=False)

#OddGames.head(7)
my_list = list(set(OddGames["Bet"]))
#OddGames = OddGames.reset_index(level=1)
#OddGames = OddGames.pivot_table(values='Bet', index=OddGames.index, columns='IdGame')
OddGames2 = OddGames.set_index(['IdGame', 'DateTime', 'Home', 'Away', 'Competition']).pivot(columns='Bet').reindex()#.rename_axis(None, axis=1)

tickers_data = []
for ticker, data in OddGames2.groupby(level=0, axis=1):
    data.columns = data.columns.droplevel(0)
    data = data.reset_index()
    data.insert(0, 'ticker', ticker)
    tickers_data.append(data)

OddGames2 = pd.concat(tickers_data).drop(['ticker'], axis=1).rename_axis(None, axis=1)
#OddGames2.columns
#OddGames2.head(7)


cols = OddGames2.columns.values
Cols = [
'IdGame', 'DateTime', 'Home', 'Away', 'Competition', 
'A', 'D', 'H',  'H/D', 'D/A', 'H/A', 
    
'H/D&M1,5', 'H/D&M2,5', 'H/D&M3,5', 'H/D&M4,5',
'H/D&P1,5', 'H/D&P2,5', 'H/D&P3,5', 'H/D&P4,5', 
    
'D/A&M1,5', 'D/A&M2,5', 'D/A&M3,5', 'D/A&M4,5',
'D/A&P1,5', 'D/A&P2,5', 'D/A&P3,5', 'D/A&P4,5', 
    
'H/A&M1,5', 'H/A&M2,5', 'H/A&M3,5', 'H/A&M4,5',
'H/A&P1,5', 'H/A&P2,5', 'H/A&P3,5', 'H/A&P4,5', 
    
'LDEM', 'LDEMP', 
'H/D&LDEM', 'H/D&LDEMP', 
'D/A&LDEM', 'D/A&LDEMP', 
'H/A&LDEM', 'H/A&LDEMP', 
    
'M0,5', 'M1,5', 'M2,5', 'M3,5', 'M4,5', 'M5,5',
'P0,5', 'P1,5', 'P2,5', 'P3,5', 'P4,5', 'P5,5']
    
OddGames3 = OddGames2[Cols].reset_index(drop=True)

WrongCol = []
for col in cols:
    if col not in OddGames3.columns.values:
        WrongCol.append(col)
        
mask = ~OddGames['Bet'].isin(Cols)
bugCols2 = OddGames[mask]['Bet'].tolist()
Teams = OddGames['Away'].unique() +  " / " +OddGames['Home'].unique()
bugCols = Check[mask]['BetOri'].tolist()
print(len(bugCols2))
print(len(bugCols))
print(Teams)
print(bugCols2)
bugCols

In [ ]:
# Save UNIBET odds
if OddGames3.isna().mean()[OddGames3.isna().mean() != 0].mean() < 0.01:
    OddGames3["Date"] = date.today().strftime("%d/%m/%Y")
    OddGames3 = OddGames3.sort_values(['IdGame'])
#     OddGames3 = OddGames3[OddGames3.columns.tolist()[-1:] + OddGames3.columns.tolist()[:-1]]

    #Save odds as CSV
    csv = pd.read_csv("Odds_Match.csv")
    def bool(p):
        #if p in pd.read_csv("Odds_NBA.csv")["id"]:
        if any(csv['IdGame'].str.contains(str(p))):
            return(False)
        else:
            return(True)

    b = OddGames3.apply(lambda p : bool(p["IdGame"]), axis=1)
    csv = pd.concat([csv,OddGames3[b]])
    csv.to_csv('Odds_Match.csv', index=False, encoding='utf-8')

In [ ]:
URL_prefix = "https://www.espn.co.uk/football/table/_/league/"
Competitions = ["ger.1", "ger.2", #Germany
                "fra.1", "fra.2", #France
                "esp.1", "esp.2", #Spain
                "ita.1", "ita.2", #Italy
                "eng.1", "eng.2", #England
                "por.1", #"por.2", #Portugal
                "sco.1", #Scotland
                "den.1", #Denmark
                "aut.1", #Austria
                "sui.1", #Suisse
                "bel.1", #Belgique
                "ned.1", #Pays-Bas
                "tur.1", #Turquie
                "gre.1", #Grèce
                "rou.1", #Roumanie
                "isr.1", #Israël
              ]

list_teams = []
for competition in tqdm(Competitions):
    u = URL_prefix + competition
    get_page(u)
    soup = BeautifulSoup(driver.page_source)
    for games in soup.findAll(class_='Table__TR Table__TR--sm Table__even'):
        for linkGame in games.findAll(class_='hide-mobile'):
            for link in linkGame.findAll(class_='AnchorLink'):
                data_item = {"Competition": competition,
                             "LinkTeam": link["href"].replace("/club/", "/team/results/").replace(re.sub(".*/", "", link["href"].replace("/club/", "/team/results/")),"")
                            }
                list_teams.append(data_item)
    for games in soup.findAll(class_='filled Table__TR Table__TR--sm Table__even'):
        for linkGame in games.findAll(class_='hide-mobile'):
            for link in linkGame.findAll(class_='AnchorLink'):
                data_item = {"Competition": competition,
                             "LinkTeam": link["href"].replace("/club/", "/team/results/").replace(re.sub(".*/", "", link["href"].replace("/club/", "/team/results/")),"")
                            }
                list_teams.append(data_item)
Teams = pd.DataFrame(list_teams)
#Teams.head()

Hist = []
Year = []
for i in tqdm(range(len(Teams))):
#for i in range(2):
    u = "https://www.espn.co.uk" + Teams.iloc[i]["LinkTeam"] + "league/" + Teams.iloc[i]["Competition"]
    get_page(u)
    soup = BeautifulSoup(driver.page_source)
    for j in soup.find_all(class_='Table__Title'):
        Year.append(j.text.split(",")[1])
    i=0
    for cat in soup.find_all(class_='Table__ScrollerWrapper relative overflow-hidden'):
        for linkGame in cat.find_all(class_='Table__TR Table__TR--sm Table__even'):
            cols = linkGame.find_all(class_='Table__TD')
            data_item = {"Date": cols[0].text + Year[i], 
                         "Home": cols[1].text,
                         "Score": cols[2].text,
                         "Away": cols[3].text
                        }
            Hist.append(data_item)
        i=i+1

In [ ]:
History = pd.DataFrame(Hist).drop_duplicates()
History[['ScoreH', 'ScoreA']] = History['Score'].str.split(' - ', expand=True).astype(int)
History[['But']] = History['ScoreH'] + History['ScoreA']
History['Result'] = np.where(History['ScoreH'] == History['ScoreA'], 'Draw',
                             np.where(History['ScoreH'] > History['ScoreA'], 'Home', 'Away'))
from datetime import datetime
# Create new variables
History['Ds'] = History['Result'].apply(lambda x: 1 if x == 'Draw' else 0)
History['Hs'] = History['Result'].apply(lambda x: 1 if x == 'Home' else 0)
History['As'] = History['Result'].apply(lambda x: 1 if x == 'Away' else 0)
History['DA'] = History['Result'].apply(lambda x: 1 if x in ['Draw', 'Away'] else 0)
History['DH'] = History['Result'].apply(lambda x: 1 if x in ['Draw', 'Home'] else 0)
History['HA'] = History['Result'].apply(lambda x: 1 if x in ['Home', 'Away'] else 0)

#History['DA'] = History['DA', ''].apply(lambda x: 1 if x == 1 else 0)
#History['DH'] = History['DH'].apply(lambda x: 1 if x == 1 else 0)
#History['HA'] = History['HA'].apply(lambda x: 1 if x == 1 else 0)

History['LDEMs'] = np.where(History['ScoreH'] > 0, np.where(History['ScoreA'] > 0, 1, 0), 0)
History['LDEMPs'] = np.where(History['LDEMs'] == 1, 0, 1)

History["Date"] = History["Date"].apply(lambda x: datetime.strptime(x,  "%a, %d %b %Y").date())
History = History.sort_values(by=["Home", "Date"], ascending=False)

#Save History until today
date_folder = os.path.join(os.getcwd() + "\HistoryV1")
os.makedirs(date_folder, exist_ok=True)
History.to_csv(os.path.join(date_folder, date.today().strftime("%Y-%m-%d") + '.csv'), index=False)   

In [ ]:
Games=10
#History = History[pd.to_datetime(History['Date']) > pd.Timestamp('2023-01-01')]

# Define the score ranges
score_ranges = list(range(1, 6))

# Loop over the score ranges and count the number of scores greater than and less than each range
for score_range in score_ranges:
    #History[f'H>{score_range}'] = (History['ScoreH'] > score_range).astype(int)
    #History[f'A>{score_range}'] = (History['ScoreA'] > score_range).astype(int)
    #History[f'H<{score_range}'] = (History['ScoreH'] < score_range).astype(int)
    #History[f'A<{score_range}'] = (History['ScoreA'] < score_range).astype(int)
    History[f'B>{score_range}'] = (History['But'] > score_range).astype(int)
    History[f'B<{score_range}'] = (History['But'] < score_range).astype(int)
    
    History[f'DHB>{score_range}'] = ((History['But'] > score_range) & (History['DH'] == 1)).astype(int)
    History[f'DHB<{score_range}'] = ((History['But'] < score_range) & (History['DH'] == 1)).astype(int)
    
    History[f'HAB>{score_range}'] = ((History['But'] > score_range) & (History['HA'] == 1)).astype(int)
    History[f'HAB<{score_range}'] = ((History['But'] < score_range) & (History['HA'] == 1)).astype(int)
    
    History[f'DAB>{score_range}'] = ((History['But'] > score_range) & (History['DA'] == 1)).astype(int)
    History[f'DAB<{score_range}'] = ((History['But'] < score_range) & (History['DA'] == 1)).astype(int)
    
    History['n'] = 1
    
HomeStats = History.groupby("Home").apply(lambda x: x.sort_values("Date", ascending=False).head(Games)).reset_index(drop=True)
HomeStats = HomeStats.groupby('Home')[
                              [f'B>{score_range}' for score_range in score_ranges] +
                              [f'B<{score_range}' for score_range in score_ranges] +
                              [f'DHB>{score_range}' for score_range in score_ranges] +
                              [f'DHB<{score_range}' for score_range in score_ranges] +
                              [f'HAB>{score_range}' for score_range in score_ranges] +
                              [f'HAB<{score_range}' for score_range in score_ranges] +
                              [f'DAB>{score_range}' for score_range in score_ranges] +
                              [f'DAB<{score_range}' for score_range in score_ranges] +
                              ['Ds'] +
                              ['Hs'] +
                              ['As'] +
                              ['DA'] +
                              ['DH'] +
                              ['HA'] +
                              ['LDEMs'] +
                              ['LDEMPs'] +
                              ['n']].sum()

HomeStats['Home'] = HomeStats.index
#HomeStats.reset_index(drop=True)
#HomeStats.index.name = None
HomeStats = HomeStats[HomeStats["n"]>=5]
HomeStats = HomeStats.set_index(['Home']).add_prefix('1').reset_index()


AwayStats = History.groupby("Away").apply(lambda x: x.sort_values("Date", ascending=False).head(Games)).reset_index(drop=True)
AwayStats = AwayStats.groupby('Away')[[#f'H>{score_range}' for score_range in score_ranges] +
                              #[f'A>{score_range}' for score_range in score_ranges] +
                              #[f'H<{score_range}' for score_range in score_ranges] +
                              #[f'A<{score_range}' for score_range in score_ranges] +
                               f'B>{score_range}' for score_range in score_ranges] +
                              [f'B<{score_range}' for score_range in score_ranges] +
                              [f'DHB>{score_range}' for score_range in score_ranges] +
                              [f'DHB<{score_range}' for score_range in score_ranges] +
                              [f'HAB>{score_range}' for score_range in score_ranges] +
                              [f'HAB<{score_range}' for score_range in score_ranges] +
                              [f'DAB>{score_range}' for score_range in score_ranges] +
                              [f'DAB<{score_range}' for score_range in score_ranges] +
                              ['Ds'] +
                              ['Hs'] +
                              ['As'] +
                              ['DA'] +
                              ['DH'] +
                              ['HA'] +
                              ['LDEMs'] +
                              ['LDEMPs'] +
                              ['n']].sum()

AwayStats['Away'] = AwayStats.index
#AwayStats.reset_index(drop=True)
#AwayStats.index.name = None
AwayStats = AwayStats[AwayStats["n"]>=5]
AwayStats = AwayStats.set_index(['Away']).add_prefix('2').reset_index()

In [ ]:
%run ./MatchTeamNames.ipynb
AwayStats["Away"].replace(merged_dict, inplace=True)
HomeStats["Home"].replace(merged_dict, inplace=True)
OddGames3["Away"].replace(merged_dict, inplace=True)
OddGames3["Home"].replace(merged_dict, inplace=True)
dfGames["Away"].replace(merged_dict, inplace=True)
dfGames["Home"].replace(merged_dict, inplace=True)

AwayStats["Away"].replace(Match_teams, inplace=True)
HomeStats["Home"].replace(Match_teams, inplace=True)
OddGames3["Away"].replace(Match_teams, inplace=True)
OddGames3["Home"].replace(Match_teams, inplace=True)
dfGames["Away"].replace(Match_teams, inplace=True)
dfGames["Home"].replace(Match_teams, inplace=True)

Match_teams

a = list(set(OddGames3["Away"])) + list(set(OddGames3["Home"]))
b = list(set(AwayStats["Away"])) + list(set(HomeStats["Home"]))
print(list(np.setdiff1d(a,b)))
print(list(np.setdiff1d(b,a)))

print(len(OddGames3))
Final = OddGames3.merge(HomeStats, on='Home', how='left').merge(AwayStats, on='Away', how='left').dropna(subset=['2As']).dropna(subset=['1Hs'])
print(len(Final))

In [2946]:

for w in weight:     
    print(w)

['50/50', 0.5, 0.5]
['2/3 1/3', 0.6666666666666666, 0.3333333333333333]
['1/3 2/3', 0.3333333333333333, 0.6666666666666666]


In [3034]:
weight

'1.0 0.0'

In [2952]:
for w in weight:  
    print(w[1]+w[2])

1.0
1.0
1.0


In [3071]:
minProba = 0.5
minDiffProba = 1.05
results_dict = {}
weight = [[f"{round(i * 0.05, 2)} {round(1 - i * 0.05, 2)}", round(i * 0.05, 2), round(1 - i * 0.05, 2)] for i in range(21)]
# weight = [["0.5 0.5", 0.5, 0.5],
#           ["0.66 0.33", 2/3, 1/3],
#           ["0.33 0.66", 1/3, 2/3]
#          ]

param_names = [['H', '1Hs', '1n', '2Hs', '2n'],
               ['D', '1Ds', '1n', '2Ds', '2n'],
               ['A', '1As', '1n', '2As', '2n'],
               
               ['H/D', '1DH', '1n', '2DH', '2n'],
               ['D/A', '1DA', '1n', '2DA', '2n'],
               ['H/A', '1HA', '1n', '2HA', '2n'],
               
               #Moins de goals
               ['H/D&M4,5', '1DHB<5', '1n', '2DHB<5', '2n'],
               ['D/A&M4,5', '1DAB<5', '1n', '2DAB<5', '2n'],
               ['H/A&M4,5', '1HAB<5', '1n', '2HAB<5', '2n'],
               
               ['H/D&M3,5', '1DHB<4', '1n', '2DHB<4', '2n'],
               ['D/A&M3,5', '1DAB<4', '1n', '2DAB<4', '2n'],
               ['H/A&M3,5', '1HAB<4', '1n', '2HAB<4', '2n'],
               
               ['H/D&M2,5', '1DHB<3', '1n', '2DHB<3', '2n'],
               ['D/A&M2,5', '1DAB<3', '1n', '2DAB<3', '2n'],
               ['H/A&M2,5', '1HAB<3', '1n', '2HAB<3', '2n'],
               
               ['H/D&M1,5', '1DHB<2', '1n', '2DHB<2', '2n'],
               ['D/A&M1,5', '1DAB<2', '1n', '2DAB<2', '2n'],
               ['H/A&M1,5', '1HAB<2', '1n', '2HAB<2', '2n'],
               
               #Plus de goals
               ['H/D&P4,5', '1DHB>4', '1n', '2DHB>4', '2n'],
               ['D/A&P4,5', '1DAB>4', '1n', '2DAB>4', '2n'],
               ['H/A&P4,5', '1HAB>4', '1n', '2HAB>4', '2n'],
               
               ['H/D&P3,5', '1DHB>3', '1n', '2DHB>3', '2n'],
               ['D/A&P3,5', '1DAB>3', '1n', '2DAB>3', '2n'],
               ['H/A&P3,5', '1HAB>3', '1n', '2HAB>3', '2n'],
               
               ['H/D&P2,5', '1DHB>2', '1n', '2DHB>2', '2n'],
               ['D/A&P2,5', '1DAB>2', '1n', '2DAB>2', '2n'],
               ['H/A&P2,5', '1HAB>2', '1n', '2HAB>2', '2n'],
               
               ['H/D&P1,5', '1DHB>1', '1n', '2DHB>1', '2n'],
               ['D/A&P1,5', '1DAB>1', '1n', '2DAB>1', '2n'],
               ['H/A&P1,5', '1HAB>1', '1n', '2HAB>1', '2n'],
               
              ]
def weighted_mean(value1, value2, weight1, weight2):
    weighted_mean = (weight1 * value1) + (weight2 * value2)
    return weighted_mean

for w in weight:        
    results = pd.DataFrame(columns=['IdGame', 'Date', 'Home', 'Away', 'Bet', 'Odd', 'MaProba', 'UnibetProba', 'DiffProba'])
    for params in param_names:
        for _, row in Final.iterrows():
            if ((row[params[2]] >= 5) and (row[params[4]] >= 5)): 
                if (w[1]+w[2]) == 1:
                    meanBet = weighted_mean(row[params[1]]/row[params[2]], row[params[3]]/row[params[4]], w[1], w[2])
                probaBet = 1/(row[params[0]]*1.05)
                if (probaBet <= meanBet):
                    results.loc[len(results)] = {'IdGame': row['IdGame'], 'Date': row['DateTime'], 'Home': row['Home'], 'Away': row['Away'], 'Bet': params[0], 
                                                'Odd': row[params[0]]*1.05, 'MaProba': meanBet, 'UnibetProba': probaBet, 
                                                'DiffProba': meanBet/probaBet
                                                }

    results = results[(results['DiffProba'] >= minDiffProba) &
                      (results['MaProba'] >= minProba)
                     ]
    results = results.sort_values('DiffProba', ascending=False)
    results_dict[f'{w[0]}'] = results

In [3076]:
from datetime import date
today = date.today()
limits = [0.8, 1]
r = results_dict["0.5 0.5"]
r = r[(r["MaProba"] >= limits[0]) & (r["MaProba"] <= limits[1])]
r[(r["Date"] <= today.strftime('%Y-%m-%d 23:59:59'))]

,IdGame,Date,Home,Away,Bet,Odd,MaProba,UnibetProba,DiffProba
680,2648409_1,2023-02-24 21:00:00,Eibar,Villarreal B,"H/A&M3,5",1.5330,0.85,0.652316,1.303050
553,2663781_1,2023-02-24 21:15:00,FC Famalicao,Portimonense,"H/A&M4,5",1.4280,0.90,0.700280,1.285200
412,2663781_1,2023-02-24 21:15:00,FC Famalicao,Portimonense,H/A,1.3125,0.95,0.761905,1.246875
472,2673615_1,2023-02-24 20:30:00,Frosinone,Parme,"H/D&M4,5",1.3125,0.95,0.761905,1.246875
610,2673615_1,2023-02-24 20:30:00,Frosinone,Parme,"H/D&M3,5",1.5330,0.80,0.652316,1.226400
289,2673615_1,2023-02-24 20:30:00,Frosinone,Parme,H/D,1.2600,0.95,0.793651,1.197000
401,2659571_1,2023-02-24 18:30:00,Karlsruhe,Jahn Regensburg,H/A,1.3125,0.90,0.761905,1.181250
1444,2659573_1,2023-02-24 18:30:00,Dusseldorf,Braunschweig,"H/D&P1,5",1.4175,0.80,0.705467,1.134000
536,2648409_1,2023-02-24 21:00:00,Eibar,Villarreal B,"H/A&M4,5",1.3335,0.85,0.749906,1.133475
236,2634371_1,2023-02-24 21:00:00,Fulham,Wolverhampton,H/D,1.3860,0.80,0.721501,1.108800


In [3038]:
#Write Day folders
for methods in weight:
    r = results_dict[methods[0]]
    r['DateD'] = pd.to_datetime(r['Date']).dt.strftime('%Y-%m-%d')

    dates = pd.to_datetime(r['Date'])
    r['Week'] = dates.dt.year.astype(str) + "W" + dates.dt.isocalendar().week.astype(str)
    r['Week'].unique()
    
    for date in r['DateD'].unique():
        date_folder = os.path.join(os.getcwd() + "\ProbaV1", date + "\\" + methods[0])
        os.makedirs(date_folder, exist_ok=True)
        date_data = r[r['DateD'] == date]
        date_data.to_csv(os.path.join(date_folder, 'data.csv'), index=False)
    
    #Write Week folders
    for date in r['Week'].unique():
        date_folder = os.path.join(os.getcwd() + "\ProbaV1", date + "\\" + methods[0])
        os.makedirs(date_folder, exist_ok=True)
        date_data = r[r['Week'] == date]
        date_data.to_csv(os.path.join(date_folder, 'data.csv'), index=False)